# Trabajo grupal


In [1]:
from math import exp, log, ceil, pi # Funciones y constantes matemáticas
from iapws import IAPWS97 # Propiedades de agua y vapor saturados.

def agua_saturada(T):
    # Devuelve un objeto con las propiedades del agua saturada a temperatura T en °C.
    return IAPWS97(T=T+273, x=0) # Calidad 0% -> Agua saturada

## Datos

#### Datos del flujo de leche

In [2]:
G_leche = 4000 # kg/h

T1 = 4 # °C
T2 = 58 # °C

u_max = 0.7 # m/s

#### Datos del calentador
Intercambiador de tubos y carcasa.

In [3]:
n_pasos_tubos = 6
n_pasos_carcasa = 1

D_int_tubos = 3/4 * 25.4e-3 # m

W_a_kcal_por_h = 60**2 / 4.182e3
h_ext = 5000 * W_a_kcal_por_h # kcal/hm^2°C

print(f'h_ext = {h_ext:,.0f} kcal/hm^2°C')

h_ext = 4,304 kcal/hm^2°C


In [4]:
Te_agua = 90 # °C

G_agua = 5.6 * agua_saturada(Te_agua).rho # kg/h

print(f'G_agua = {G_agua:,.0f} kg/h')

G_agua = 5,406 kg/h


#### Suposiciones generales
Para el rango de temperaturas del ejercicio se considera despreciable la variación $C_p$ del agua con la temperatura y se asume:
$$C_p = 1 kcal/kg^{\circ}C$$

Debido a esto si $C$ y $G$ son los valores numéricos de la capacitancia térmica y el gasto de fluido repectivamente, se tiene que:
$$ C = G $$

En los cálculos que siguen se usa indistintamente $G$ en lugar de $C$, considerando que la conversión de unidades está implícita.

### Parte A
Problema de diseño.  
Leche pasando por tubos y agua por carcasa.

In [5]:
T3 = 72 # °C (Dato)

In [6]:
Tm_leche = (T2 + T3) / 2

leche = agua_saturada(Tm_leche) # Propiedades de la leche asimilables a las del agua

Del balance al agua caliente se tiene que la temperatura de salida es:

In [7]:
Ts_agua = Te_agua - G_agua / G_leche * (T3 - T2)

print(f'Ts_agua = {Ts_agua:.1f} °C')

Ts_agua = 71.1 °C


De los datos del calentador se calculan la cantidad de tubos por paso, la cantidad total de tubos y la velocidad de la leche dentro de los mismos:

In [8]:
A_int_tubos = pi / 4 * D_int_tubos**2

G_max_por_tubo = A_int_tubos * u_max * leche.rho * 60**2

n_tubos_por_paso = ceil(G_leche / G_max_por_tubo)

print(f'n_tubos_por_paso = {n_tubos_por_paso:.0f}')

n_tubos_por_paso = 6


In [9]:
n_total_tubos = n_tubos_por_paso * n_pasos_tubos

print(f'n_total_tubos = {n_total_tubos:.0f}')

n_total_tubos = 36


In [10]:
u_leche = G_leche / (n_tubos_por_paso * leche.rho * A_int_tubos) / 60**2

print(f'u_leche = {u_leche:.2f} m/s')

u_leche = 0.66 m/s


Como $G_{\text{leche}} < C_{\text{agua}}$ se tiene:
$$ C_{\text{min}} = C_{\text{leche}} $$
Y la eficiencia del intercambiador es:

In [11]:
q_max = G_leche * (Te_agua - T2)

q = G_leche * (T3 - T2)

eff = q / q_max

print(f'eff = {eff:.1%}')

eff = 43.8%


In [12]:
Cr = G_leche / G_agua

print(f'Cr = {Cr:.2f}')

Cr = 0.74


Utilizando la fórmula del Incropera para el $NTU$ de un intercambiador de tubos y carcasa con un pase en carcasa se tiene:

In [13]:
# Variables auxiliares
F = (1 + Cr**2)**(1/2)
E = (2 / eff - (1 + Cr)) / (F)

NTU = - F**-1 * log((E - 1) / (E + 1))
print(f'NTU = {NTU:.2f}')

NTU = 0.76


#### Cálculo del coeficiente global de transferencia de calor $U$
##### Cálculo de $h_{\text{leche}}$

In [14]:
Re_leche = u_leche * D_int_tubos / leche.nu

print(f'Re_leche = {Re_leche:.2e}')

Re_leche = 2.85e+04


Por lo tanto el flujo es turbulento.

Asumiendo:
* Tubos lo suficientemente largos.
* Espesor de pared despreciable.
* Debido a que los fluidos son asimilables y los gastos son comparables. Se asume un valor medio para la temperatura de la pared.

In [15]:
Tm_leche = (T2 + T3) / 2
Tm_agua = (Te_agua + Ts_agua) / 2

Ts = (Tm_leche + Tm_agua) / 2

In [16]:
Prs = agua_saturada(Ts).Prandt

In [17]:
Nu = 0.021 * Re_leche**0.8 * leche.Prandt**0.43 * (Prs / leche.Prandt)**0.25

h_leche = Nu * leche.k / D_int_tubos * W_a_kcal_por_h 

print(f'h_leche = {h_leche:,.0f} kcal/hm^2°C')

h_leche = 3,433 kcal/hm^2°C


In [18]:
U = (h_leche**-1 + h_ext**-1)**-1

print(f'U = {U:,.0f} kcal/hm^2°C')

U = 1,910 kcal/hm^2°C


Utilizando la definición de $NTU$ se calcula el área total de transferencia para los tubos y luego el largo de los mismos:

In [19]:
A_total = NTU * G_leche / U

print(f'A_total = {A_total:.2f} m^2')

A_total = 1.59 m^2


In [20]:
L = A_total / (n_total_tubos * pi * D_int_tubos)

print(f'L = {L:.2f} m')

L = 0.74 m


### Parte B

Se desprecia el espesor del serpentín.

In [21]:
Tamb = 20 # °C (Dato)
h_amb = 10 * W_a_kcal_por_h # kcal/hm^2°C (Dato)

u_serpentin = 0.7 # m/s (Dato)

t_paust = 20 # s (Dato)

In [22]:
leche_serpentin = agua_saturada(T3)

De los datos se calculan el diámetro y largo del serpentin:

In [23]:
A_int_serpentin = G_leche / (leche_serpentin.rho * u_serpentin * 60**2)
D_serpentin = (4 / pi * A_int_serpentin)**(1/2)

print(f'D_serpentin = {D_serpentin:.3f} m')

D_serpentin = 0.045 m


In [24]:
L_serpentin = t_paust * u_serpentin

print(f'L_serpentin = {L_serpentin:.1f} m')

L_serpentin = 14.0 m


Se verifica la suposición de tubos largos.  
Debido a que $h_{\text{amb}}<<h_{\text{leche}}$ se desprecia el efecto de $h_{leche}$ en $U$ y se asume:
$$U = h_{amb}$$

In [25]:
A_serpentin = pi * D_serpentin * L_serpentin
q_serpentin = h_amb * A_serpentin * (T3 - Tamb)

T4 = T3 - q_serpentin / G_leche

caida_de_temperatura = T3 - T4

print(f'caida_de_temperatura = {caida_de_temperatura:.1f} °C')

caida_de_temperatura = 0.2 °C


Para analizar si traería problemas utilizar un caño de diámetro más pequeño, consideramos los efectos que tendría usar dicho caño:
1. Aumenta $h_{\text{leche}}$
2. Aumenta $L_\text{serpentin}$

Como se epxlicó anteriormente, el efecto de $1.$ es despreciable ya que $h_{\text{amb}}<<h_{\text{leche}}$.  
Se analiza el efecto de $2.$ calculando el nuevo largo necesario para el serpentin y la nueva caida de temperatura.:

In [26]:
D_serpentin = 0.045 / 2

A_int_serpentin = pi / 4 * D_serpentin**2
u_serpentin = G_leche / (A_int_serpentin * leche_serpentin.rho * 60**2)

print(f'u_serpentin = {u_serpentin:.2f} m/s')

u_serpentin = 2.86 m/s


In [27]:
L_serpentin = t_paust * u_serpentin

print(f'L_serpentin = {L_serpentin:.1f} m')

L_serpentin = 57.2 m


In [28]:
A_serpentin = pi * D_serpentin * L_serpentin
q_serpentin = h_amb * A_serpentin * (T3 - Tamb)

T4 = T3 - q_serpentin / G_leche

caida_de_temperatura = T3 - T4

print(f'caida_de_temperatura = {caida_de_temperatura:.1f} °C')

caida_de_temperatura = 0.5 °C


La caída de temperatura de la leche está dentro de lo admisible incluso considerando un serpentín con diámetro igual a la mitad del considerado originalmente.

### Parte C

Para el intercambiador regenerativo se tiene $C_r = 1$ y la eficiencia es:

In [29]:
q_max = G_leche * (T3 - T1)

q = G_leche * (T2 - T1)

eff = q / q_max

print(f'eff = {eff:.1%}')

eff = 79.4%


Viendo los ábacos de $NTU$ en función de $C_r$ y la eficiencia se puede ver que la única configuración viable es la de contraflujo.
Las demás configuraciones no alcanzarían la eficiencia obtenida para $C_r = 1$.

### Parte D
#### Cálculo de $T_2$

No se indica el tipo de intercambiador.  
Se asume un intercambiador de tubo en tubo conectado (según parte C) en contraflujo.  
Para otro tipo de intercambiador el método de resolución es el mismo. Solo es necesario utilizar la correlación $\epsilon-NTU$ apropiada.  

Como la caída de temperatura en el serpentín es pequeña, despreciamos el cambio en $T_4$ al disminuír el caudal.

Utilizando los resultados en la parte anterior, calculamos $UA$ para el intercambiador regenerativo.  
La fórmula para $NTU$ con $C_r = 1$ es:

In [30]:
NTU = eff / (1 - eff)

print(f'NTU = {NTU:.2f}')

NTU = 3.86


In [31]:
UA = NTU * G_leche

print(f'UA = {UA:,.0f} kcal')

UA = 15,429 kcal


Ahora analizamos la situación al disminuír el caudal de leche:

In [32]:
G_leche = 4000 * 0.9 # kg/h

Como los coeficientes de convección que determinan $U$ corresponden ambos a flujo interno en régimen turbulento, se tiene:
$$ h \propto {u_{\text{leche}}}^{0.8} $$

Como $u_{\text{leche}} \propto G_{\text{leche}}$ se tiene:
$$ h \propto {G_{\text{leche}}}^{0.8} $$

Por lo que:
$$ U \propto {G_{\text{leche}}}^{0.8} $$

Entonces, podemos calcular el nuevo coeficiente global de transferencia de la siguiente manera:
$$ U_{\text{actual}} = U_{\text{anterior}} \Big( \frac{G_{\text{actual}}}{G_{\text{anterior}}} \Big)^{0.8} $$


In [33]:
# UA = 15429 kcal/h°C calculado anteriormente
UA = UA * 0.9**0.8

print(f'UA = {UA:,.0f} kcal')

UA = 14,181 kcal


Entonces, $NTU$ vale:

In [34]:
NTU = UA / G_leche

print(f'NTU = {NTU:.2f}')

NTU = 3.94


In [35]:
eff = NTU / (1 + NTU)

print(f'eff = {eff:.1%}')

eff = 79.8%


In [36]:
q_max = G_leche * (T4 - T1)

q_regenerativo = eff * q_max

print(f'q_regenerativo = {q_regenerativo:,.0f} kcal/h')

q_regenerativo = 193,938 kcal/h


In [37]:
T2 = T1 + q_regenerativo / G_leche

print(f'T2 = {T2:.1f} °C')

T2 = 57.9 °C


#### Cálculo de la nueva temperatura de entrada del agua en el calentador
Se desprecia la variación en $T_s$.  
Para el nuevo coeficiente de convección de la leche se tiene:

In [38]:
# h_leche = 3433 kcal/hm^2°C (Calculado anteriormente)
h_leche = h_leche * 0.9**0.8

In [39]:
U_calentador = (h_leche**-1 + h_ext**-1)**-1

print(f'U_calentador = {U_calentador:,.0f} kcal/hm^2°C')

U_calentador = 1,821 kcal/hm^2°C


Utilizando este valor para $U$ y el valor del área de intercambio del calentador calculado anteriormente podemos calcular los $NTU$:

In [40]:
Cr = G_leche / G_agua

print(f'Cr = {Cr:.2f}')

Cr = 0.67


In [41]:
NTU = U_calentador * A_total / G_leche

print(f'NTU = {NTU:.2f}')

NTU = 0.80


De la fórmula del Incropera para la eficiencia de un intercambiador de tubos y carcasa con un pase en carcasa:

In [42]:
# Variables auxiliares
E = (1 + Cr**2)**(1/2)
F = exp(-NTU * E)

eff = 2 * (1 + Cr + E * (1 + F) / (1 - F))**-1

print(f'eff = {eff:.1%}')

eff = 46.0%


De la eficiencia y las temperaturas de la leche, podemos calcular la nueva temperatura de entrada del agua:

In [43]:
q_calentador = G_leche * (T3 - T2)

q_max_calentador = q_calentador / eff

Te_agua = T2 + q_max_calentador / G_leche

print(f'Te_agua = {Te_agua:.1f} °C')

Te_agua = 88.6 °C


La temperatura del agua caliente se mantiene igual.  